---
title: Lab 1 Initial Page
author: Zoe Worrall  - zworrall@g.hmc.edu
date: September 2, 2024
\contact: "Contact at zworrall@g.hmc.edu"

format:
  html:
    other-links:
      - text: "E155 FA Development Board"
        href: https://pages.hmc.edu/brake/class/e155/fa23/assets/doc/E155%20Development%20Board%20Schematic.pdf
      - text: "Lab 1 Github Files"
        href: https://github.com/zoe-worrall/e155_labs/tree/main/fpga/radiant_project/lab01
      - text: "Tutorial for Programming FPGAs"
        href: running_labs.html

categories:
  - labreport
---




## Lab 1: FPGA and MCU Setup and Testing

### Introduction
The goal of this lab was initial setup and confirmation of FPGA control of on-board LEDs and attached GPIO pin. This was done through the control of three leds: **D6**, **D7**, and **D8**, as well as seven GPIO pins: **45**, **3**, **44**, **9**, **13**, **12**, and **20** through the readout of on-board 4-pin DIP switch. By the end of the laboratory, the three switches were controlled as defined as seen in Tables 1(a) and 1(b), and the seven segment LED matrix as defined in Table 2.

::: {.grid}

::: {.g-col-6}

::: {#tbl-panel layout-nrow=2}
| s[3] | s[2] | led[1] |
|------|------|--------|
| 0    | 0    | 0      |
| 0    | 1    | 0      |
| 1    | 0    | 0      |
| 1    | 1    | 1      |

: LED 1 (D7) {#tbl-first}

| s[1] | s[0] | led[0] |
|------|------|--------|
| 0    | 0    | 0      |
| 0    | 1    | 1      |
| 1    | 0    | 1      |
| 1    | 1    | 0      |

: LED 0 (D6) {#tbl-second}

**LED Logic for Table** 

:::
*The logic tables that were used for the led to switch mapping. @tbl-first refers to s[3] and s[2]; on the board, these are switches 1 and 2 (pins P32 and P31 respectively) to match where the highest order digits would be in a four digit number. By extensino, @tbl-second uses the two right-most switches (P35 and P32)*

:::

::: {.g-col-6}
| Switch Base 10 Value | Seven Segment Display |
|---------------------------|-----------------------------|
|            0              |             0               |
|            1              |             1               |
|            2              |             2               |
|            3              |             3               |
|            4              |             4               |
|            5              |             5               |
|            6              |             6               |
|            7              |             7               |
|            8              |             8               |
|            9              |             9               |
|            10             |             A               |
|            11             |             B               |
|            12             |             C               |
|            13             |             D               |
|            14             |             E               |
|            15             |             F               |

: **Seven Segment LED Matrix Logic** {#tbl-letters}

*The mapping from the decimal readout of the switches in hexadecimal.*
:::


:::

### Hardware Setup

The first stage of this lab involved the soldering of the MCU and the FPGA, as well as all additional SMD and THT pins. To do this, the [E155 FA23 Development Board](https://pages.hmc.edu/brake/class/e155/fa23/assets/doc/E155%20Development%20Board%20Schematic.pdf) schematic was used. To begin, the SMD components were soldered. Careful consideration was given to make sure that the diodes were soldered in the right order and facing the correct direction. After these were soldered, THT components were then soldered, including the 4-pin DIP switch, the 8-pin DIP switch, and the female headers (@fig-imag1).

![SMD and THT pins soldered](images/SMD_and_THT.JPG){#fig-imag1  width=50% fig-align="center" fig-alt="Image 1"}


A board was connected via ribbon cable to the main Development board to allow for control of a breadboard via on-board GPIO pins (@fig-imag2). Once all pieces had been put together, the board was tested via a voltmeter via the screw terminal to ensure that the board could be properly powered (@fig-imag3)(@fig-imag4). The board through the screw terminal required +6 - +20 V; testing was done using 6V, and the board was confirmed to be running. The breadboard was tested using a multimeter to ensure that the 3.3V, 5.0V, and GND pins were properly generating the right voltage. 

::: {.grid}

::: {.g-col-4}

::: {#fig-imag2}

<iframe src="images/Full_Board_Assembled.JPG" fig-align="center" fig-alt="Image 2"></iframe>
Board assembled with ribbon cable

:::

:::

::: {.g-col-4}

::: {#fig-imag3}
<iframe src="images/Powered_By_ScrewTerminal.JPG" fig-align="center" fig-alt="Image 3"></iframe>
Powering the board with the screw terminal at +6V

:::

:::

::: {.g-col-4}

::: {#fig-imag4}

<iframe src="images/PowerScrewTerminal.JPG" fig-align="center" fig-alt="Image 4"></iframe>
The Voltmeter while powering the screw terminal

:::

:::

:::

All soldered pins and the required impedance values and relevant pin numbers can be found in the E155 FA23 Development Board pdf, linked in "Other Links" on this page. Once confirmation was had that the board was properly powering all pins using the screw terminal, the transition was then made to powering the board using both the MCU and the FPGA individually via a micro-USB cable.

### Setup for Lab

I own a Mac computer, so I needed to go through a broad array of necessary preparation in order to begin generating the code for my computer. First, I installed SEGGER Embedded Studio 8.16a on my computer. Following the [SEGGER Embedded Studio Tutorial](https://hmc-e155.github.io/tutorials/segger-embedded-studio-setup/index.html), I confirmed that I was able to load designs onto my MCU board. 

::: {.callout-note title="Mac Users Aid"} 

Unfortunately, because I don't have a Windows, it was necessary to run to the laboratory and use an actual Windows computer in order to install J-Link. The only thing you need to download is the third link on the [download links](https://www.segger.com/products/debug-probes/j-link/models/other-j-links/st-link-on-board/), specifically the one called "SEGGER STLinkReflash utility". You shouldn't need to install anything additional, since the supplied boards already had J-Link installed.

:::

The FPGA proved to be a little more difficult. This was a multiple step process, and required a bit of troubleshooting.

First, I installed [Parallels](https://www.parallels.com) on my computer: I know that not everyone has $80 lined up to pay for a virtual Windows Machine. I recommend, if possible, use a student discount; when I got it, I managed to find a 50% student discount, which meant in the end I only spent $40. Parallels is one of the best ways to interact with your FPGA besides being in-lab, and I highly recommend it after using it for the past few days.

Once I had that downloaded, I installed [Lattice Radiant](https://www.latticesemi.com/Products/DesignSoftwareAndIP/FPGAandLDS/Radiant) for Windows 64 inside of the Parallels machine. In all, this process took me about two hours, so make sure to budget some time beforehand if you're planning on doing this on a Mac. As noted by Professor Brake on [his page explaining how to use Lattice Radiant](https://hmc-e155.github.io/tutorials/lattice-radiant-installation/index.html), if you build a design within this program you won't be able to run it like you would on a Windows, even if you are able to Synthesize the board. Instead, you need to go back to your home screen and find the correct pathing to the file on your Windows computer. 


::: {.callout-note title="Mac Users Aid"} 

Not many people seem to know this, but if you have a folder or file open in your File Manager, you can drag the file into an open terminal window in order to get the complete file path; I'm sure there are other ways to do this, but this is the fastest way for me to open most programs.

:::

Although you could run the same terminal code that Professor Brake advises, I personally have been using code given to my by Kavi Dey, which works better most of the time because I don't need to figure out pathing to the .bin file on my computer. If you're interested, follow [this tutorial](running_labs.qmd).

### The Lab Itself

The lab is composed of three modules, one test bench, and one basic text file of test vectors. The modules are assumbled such that there is one for the combinational logic used for the LEDs called `led_control.sv`, and another that is used for the combinational logic of the seven segment array, called `seven_segments.sv`. The connection of these modules can be visualized in the block diagram below.

#### INSERT BLOCK DIAGRAM


In this design approach, I noted that the led's followed very basic boolean logic; LED 0 followed XOR logic with the lowest order two bits supplied by the DIP switch, and LED 1 followed AND logic with the two highest bits. LED 2 was very simple to program, as it followed similar logic to that used in the [iCE_40.sv tutorial.](https://hmc-e155.github.io/tutorials/lattice-radiant-ice40-ultraplus-project-setup/).

The design approach for the seven segment array required some more thoughtful setup. First, I constructed a table for each of the 16 possible arrangements of the DIP switch. Following the general logic that 0 should be mapped to 0x0, 1 to 0x1, etc. etc. until 15 was mapped to 0xF, I then created a table @tbl-thisTabl that mapped out which parts of the segment would need to turn on to display the correct number. This gave me a list of seven bits arrange in 16 ways, which I then used inside a case statement when assigning according to the decimal value held by the DIP switch. Note that in the following table, segA was seg[0], or the least significant bit of the seven_segment array inside of the module; I initially had an issue debugging where I reversed the values on accident, and it created some odd shapes on the seven-segment matrix.


In [ ]:
#| label: tbl-planet-measures
#| tbl-cap: Astronomical object

import tabulate

from IPython.display import Markdown
from tabulate import tabulate
table = [[0,0,0,0,  0,0,0,0,0,0,0,1],
         [0,0,0,1,  1,1,0,0,1,1,1,1],
         [0,0,1,0,  2,0,0,1,0,0,1,0],
         [0,0,1,1,  3,0,0,0,0,1,1,0],
         [0,1,0,0,  4,1,0,0,1,1,0,0],
         [0,1,0,1,  5,0,1,0,0,1,0,0],
         [0,1,1,0,  6,0,1,0,0,0,0,0],
         [0,1,1,1,  7,0,0,0,1,1,1,1],
         [1,0,0,0,  8,0,0,0,0,0,0,0],
         [1,0,0,1,  9,0,0,0,0,1,0,0],
         [1,0,1,0,  "A",0,0,0,1,0,0,0],
         [1,0,1,1,  "b",1,1,0,0,0,0,0],
         [1,1,0,0,  "C",0,1,1,0,0,0,1],
         [1,1,0,1,  "d",1,0,0,0,0,1,0],
         [1,1,1,0,  "E",0,1,1,0,0,0,0],
         [1,1,1,1,  "F",0,1,1,1,0,0,0]]
Markdown(tabulate(
  table, 
  headers=["s[3]", "s[2]", "s[2]", "s[0]", "0x___", "segA", "segB", "segC", "segD", "segE", "segF", "segG"]
))

Brief (e.g., 3-5 sentence) description of the main goals of the assignment and what was done.
Explanation of design approach. How did you go about designing and implementing the design?
Explanation of testing approach. How did you verify your design was behaving as expected?
Statement of whether the design meets all the requirements. If not, list the shortcomings.
Number of hours spent working on the lab are included.
Writeup contains minimal spelling or grammar issues and any errors do not significantly detract from clarity of the writeup.
(Optional) List commments or suggestions on what was particularly good about the assignment or what you think needs to change in future versions.


## The basic blueprint plan

PDF insert of the lab report will go here